<a href="https://colab.research.google.com/github/baselfirb/TextMining/blob/master/%E7%B5%8C%E6%B8%88%E3%83%AC%E3%83%9D%E3%83%BC%E3%83%88%E4%BD%9C%E6%88%90_v085.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 使用方法
## 1. レポート作成範囲を設定
- 開始年月、作成期間を入力する

## 2.対象国・地域を指定
- JP : 日本
- US : アメリカ
- EU : EU
- AU : オーストラリア

# レポート作成　初期設定

In [ ]:
# 開始年月
start_ym = "2025年4月"

# 期間(月数)：開始年月を含む期間指定
# 6か月：7, ３か月：4 を入力
n_term = 4

# 対象国 / 地域
area = "JP"

# ライブラリインポート

In [3]:
import os
import requests
import time
import pathlib as Path
from dotenv import load_dotenv
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from docx import Document as DocxDocument

from bs4 import BeautifulSoup
import pdfplumber
import logging

import tiktoken
from langchain_openai import ChatOpenAI

from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain.chains.summarize import load_summarize_chain


from langchain.schema import Document
from langchain.utilities import GoogleSerperAPIWrapper

ModuleNotFoundError: No module named 'pdfplumber'

In [4]:
%pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 44.8 MB/s eta 0:00:00


# 変数設定

## 環境変数

In [ ]:
# .envファイルの読み込み
load_dotenv()

# Open AI API
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Serper API
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# Deepl API
DEEPL_API_KEY = os.getenv("DEEPL_API_KEY")  # DeepL APIキーは環境変数から取得

# ユーザーエージェント
USER_AGENT = os.getenv("USER_AGENT", "rag-financial-reporter/1.0")

# TPM値
TPM_LIMIT = os.getenv("OPENAI_TPM_LIMIT")

## モデル関連

In [ ]:
# モデル指定
model = "gpt-4o-mini"

# embedding_model
embedding_model = "text-embedding-3-small"

# 最大トークン数
max_token = 128000

# 抽出するURL数
url_num = 10

# 検索ドキュメントの最小文字数
# 注：指定した文字数に達しない検索結果は採用されない
min_docs = 500

# チャンクサイズ
chunk_sizes = 1000

# チャンクのオーバーラップ文字数
overlap = 300

# 回答の安定性（再現性）
temp = 1.0

## LLM インスタンス設定

In [ ]:
# LLM インスタンス設定
llm = ChatOpenAI(model_name=model, temperature=temp)

## その他

In [ ]:
# pdfminer.six のロガーの出力レベルを ERROR に設定（警告や情報を出さない）
logging.getLogger("pdfminer").setLevel(logging.ERROR)

In [ ]:
# レポート作成対象の国と中央銀行などの辞書
area_dict = {
    "JP": ("日本", "日本銀行", "BOJ", "Japan"),
    "US": ("アメリカ", "連邦準備制度理事会", "FRB FOMC", "the US"),
    "EU": ("欧州", "欧州中央銀行", "ECB", "the EU"),
    "AU": ("オーストラリア", "オーストラリア準備銀行", "RBA", "Australia"),
}

# 実行日取得
today = datetime.now().strftime("%Y-%m-%d")

# レポート作成関数

## 年月リスト作成

In [ ]:
def get_ym_list(s_ym, n_term):

    # 開始年月から n_term 分だけずらしたリストを返す
    return sorted([(datetime.strptime(s_ym, "%Y年%m月") + relativedelta(months=i)).strftime("%Y年%m月") for i in range(n_term)])

## Promt作成

In [ ]:
def make_prompt(month, area):

    # 対象国のマップを取得
    name = area_dict.get(area)

    # area に対応した prompt を返す
    return f"""
以下は{month}の{name[0]}経済に関する月次レポートを作成するための指示です。

Step1: {month}における{name[1]}（{name[2]}）の金融政策決定会合の内容とその背景について教えてください。（会合が行われなかった場合は、「会合なし」と記述）
Step2: {month}時点での{name[0]}のインフレ率、経済成長率、長期金利などの経済環境を説明してください。
Step3: 上記の{name[2]}の決定内容と経済環境を踏まえて、{month}の{name[0]}経済全体を1段落で要約してください。
"""

## URL検索クエリ作成

In [ ]:
def make_search_query_jp(month, area):

    # 対象国のマップを取得
    val = area_dict.get(area)

    # area に対応した 検索クエリ を返す
    return f"{month} {val[1]} {val[2]} 金融政策 {val[0]}経済 インフレ CPI"

In [ ]:
def make_search_query_en(month, area):

    # 対象国のマップを取得
    val = area_dict.get(area)

    # datetimeに変換（日本語表記から）
    month_dt = datetime.strptime(month, "%Y年%m月")

    # 英語表記でフォーマット
    month_en = month_dt.strftime("%Y %B")

    # area に対応した 検索クエリ を返す
    return f"{month_en} {val[2]} Monetary Policy Decisions,  {val[3]} Economy,  Inflation, CPI, economic growth rate"

## Wordで保存

In [ ]:

def save_report_docx(reports, area, chain):

    # 月別サマリを初期化
    monthly_report =[]

    # 対象国のマップを取得
    name = area_dict.get(area)

    # ファイル名設定
    filename = f"{name[2]}経済レポート_{today}_{chain}_{model}.docx"

    # Word文書の新規作成
    doc = DocxDocument()

    # 表題 level:0
    doc.add_heading("経済環境（月次）", 0)

    doc.add_paragraph(f"日付: {today}")

    #　回答をWord書式で書き込み
    for month, monthly_report, urls in reports:

        # 表題 level:1
        doc.add_heading(f"対象月：{month}", level=1)

        # 回答本文の書き込み
        doc.add_paragraph(monthly_report)

        # 表題 level:2
        doc.add_heading("参考URL一覧", level=2)

        # 参考URLの書き込み
        for url in urls:

            # 参考URLの書き込み
            doc.add_paragraph(url)

        # 改ページ設定
        doc.add_page_break()

    # Wordで保存
    doc.save(filename)

    print(f"レポート保存完了: {filename}")

## URL検索

In [ ]:
def search_url(query_text):

    # 'query_text' にヒットするURLを取得
    res_url = GoogleSerperAPIWrapper().results(query_text)


    return [r["link"] for r in res_url.get('organic', [])[:url_num] if r.get('link')]

## テキスト抽出

In [ ]:


def text_from_url(url):
    try:
        # ヘッダー設定＋タイムアウト
        res = requests.get(url, headers={"User-Agent": USER_AGENT}, timeout=10)

        # HTTPエラー検出
        res.raise_for_status()

        # 文字化け対策
        if not res.encoding:

            # 明示的なエンコーディングがなければUTF-8に設定
            res.encoding = "utf-8"

        # PDFの場合
        if url.lower().endswith(".pdf"):

            temp_pdf = "temp.pdf"

            # 一時的に保存
            with open(temp_pdf, "wb") as f:
                f.write(res.content)

            text = ""

            # pdfplumberで読み取り
            with pdfplumber.open(temp_pdf) as pdf:

                for page in pdf.pages:

                    text += page.extract_text() or ""

            # 不要なファイルは削除
            os.remove(temp_pdf)

            return Document(page_content=text.strip(), metadata={"source_url": url})

        # HTMLの場合
        else:
            soup = BeautifulSoup(res.text, "html.parser")
            text = soup.get_text(separator="\n", strip=True)
            return Document(page_content=text.strip(), metadata={"source_url": url})

    except Exception as e:
        print(f"読み込み失敗: {url} -> {e}")


## 最大トークン制限

In [ ]:
def limit_tokens(documents, max_tokens=max_token, model=model):

    """
    documents : build_docs_from_query関数で分割されたテキスト
    """

    # 指定した model に適合するエンコーダー取得
    enc = tiktoken.encoding_for_model(model)

    # documents のリスト初期化
    limit_docs = []

    # トークン数初期化
    total_tokens = 0

    # 各 documents に対してトークン数を計算し、トークン制限内で documents テキストを追加
    for doc in documents:

        # 各 documents の テキストをエンコードし、トークン数を取得
        tokens = len(enc.encode(doc.page_content))

        # トークン制限を超えるまで、テキストを追加
        if total_tokens + tokens > max_tokens:

            break

        # テキストを追加
        limit_docs.append(doc)

        # トークン数を更新
        total_tokens += tokens

    print(f"トークン合計 {total_tokens} / 最大 {max_tokens}")

    # トークン制限内のテキストを返す
    return limit_docs

## ChromaDBに格納するRAG構築

In [ ]:
def build_vectorstore_from_query(search_query_jp, search_query_en, persist_dir):

    # 日本語のテキストを入れる箱を用意
    documents_jp = []

    # 英語のテキストを入れる箱を用意
    documents_en = []

    # 日本語URL検索
    urls_jp = search_url(search_query_jp)

    # 英語URL検索
    urls_en = search_url(search_query_en)

    print(f"日本語 取得URL件数 : {len(urls_jp)}")
    print(f"英語 取得URL件数 : {len(urls_jp)}")

    # 日本語URL検索結果からテキスト抽出
    documents_jp = [text_from_url(url) print(url) for url in urls_jp]
    # 英語URL検索結果からテキスト抽出
    documents_en = [text_from_url(url) print(url) for url in urls_en]


    # 最低文字数以上のテキストを抽出できれば、テキストを追加する
    documents_jp = [doc for doc in documents_jp if doc and len(doc.page_content) > 500]
    documents_en = [doc for doc in documents_en if doc and len(doc.page_content) > 500]

    # テキストがない場合
    if not documents_jp and not documents_en:
        raise ValueError("有効な文書が取得できませんでした。")


    print("***** テキスト分割　開始 *****")
    # テキスト分割器
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_sizes,
        chunk_overlap=overlap,
        separators=["\n\n", "\n" ,"。", "." , "、", ",",  " ", ""] # 段落, 改行, 句読点, スペース
    )

    # 日本語テキストを分割
    split_jp = splitter.split_documents(documents_jp)
    # 英文テキストを分割
    split_en = splitter.split_documents(documents_en)


    # 最大トークンまでテキストを保持
    doc_jp = limit_tokens(split_jp, max_tokens=max_token)
    doc_en = limit_tokens(split_en, max_tokens=max_token)

    print("***** ベクトル化　開始 *****")
    # テキスト結合
    combined_docs = doc_jp + doc_en

    # 結合してベクトル化
    embedding = OpenAIEmbeddings(model=embedding_model)

    # ChromaDBに保存
    vectordb = Chroma.from_documents(combined_docs, embedding, persist_directory=persist_dir)

    # すべての出典URL
    all_urls = [doc.metadata["source_url"] for doc in documents_jp + documents_en if "source_url" in doc.metadata]

    return vectordb, all_urls


SyntaxError: invalid syntax. Perhaps you forgot a comma? (2080308613.py, line 19)

## べクトルDBから回答を生成

In [ ]:

def answer_from_vectordb_stuff_confirm(query: str, vectordb, llm):

    # ドキュメント検索（上位10件）
    retriever = vectordb.as_retriever(search_kwargs={"k": 10})

    # クエリから関連するテキストを抽出
    docs = retriever.invoke(query)

    # プロンプトを「比較・整合性判断を促す」よう修正
    prompt = PromptTemplate.from_template(
        """以下は複数の情報ソースから得られた抜粋です。

{context}

これらを比較・検討したうえで、質問に対して最も信頼できる答えを日本語で簡潔に述べてください。
情報に矛盾がある場合は、整合性を考慮して、もっとも一貫性のある情報を採用してください。
情報が不十分な場合はその旨を明記してください。

質問: {question}
"""
    )

    # stuff型チェーン構築
    qa_chain = (
        RunnableLambda(lambda docs: {
            "context": "\n\n".join([doc.page_content for doc in docs]),
            "question": query
        })
        | prompt
        | llm
        | StrOutputParser()
    )

    return qa_chain.invoke(docs)


### stuff

In [ ]:
def answer_from_vectordb(query: str, vectordb):

    # 上位 n 件の類似文書を取得
    retriever = vectordb.as_retriever(search_kwargs={"k": 10})

    docs = retriever.invoke(query)

    # プロンプト（context + question）
    prompt = PromptTemplate.from_template(
        "以下の情報に基づいて質問に答えてください。\n\n{context}\n\n質問: {question}"
    )

    # stuffチェーンの構築
    qa_chain = (
        RunnableLambda(lambda docs: {
            "context": "\n\n".join([doc.page_content for doc in docs]),
            "question": query
        })
        | prompt
        | llm
        | StrOutputParser()
    )

    return qa_chain.invoke(docs)

### map_reduce

In [ ]:

def answer_from_vectordb_map(query: str, vectordb):

    # 上位 n 件の類似文書を取得
    retriever = vectordb.as_retriever(search_kwargs={"k": 10})

    relevant_docs = retriever.invoke(query)

    # mapステップ：各文書を要約または質問に回答させる
    map_prompt = PromptTemplate.from_template(
        "以下の文書に基づいて、質問に答えてください：\n\n{context}\n\n質問: {question}"
    )
    map_chain = map_prompt | llm | StrOutputParser()

    # reduceステップ：複数の回答を統合する
    reduce_prompt = PromptTemplate.from_template(
        "以下は複数の文書に基づく回答候補です。これらを統合し、最も適切な回答を作成してください：\n\n{context}\n\n質問: {question}"
    )
    reduce_chain = reduce_prompt | llm | StrOutputParser()

    # map-reduceチェーンの構築
    qa_chain = (
        RunnableLambda(lambda docs: [{"context": doc.page_content, "question": query} for doc in docs])
        | map_chain.map()
        | (lambda results: {"context": "\n\n".join(results), "question": query})
        | reduce_chain
    )


    return qa_chain.invoke(relevant_docs)


### refine

In [ ]:
#from langchain.prompts import PromptTemplate
#from langchain_core.output_parsers import StrOutputParser
#from langchain_openai import ChatOpenAI

def answer_from_vectordb_ref(query: str, vectordb):
    retriever = vectordb.as_retriever(search_kwargs={"k": 10})
    docs = retriever.invoke(query)

    llm = ChatOpenAI(model_name=model, temperature=temp)

    # 初期回答を生成するプロンプト
    initial_prompt = PromptTemplate.from_template(
        "以下の文書に基づいて、質問に答えてください。\n\n{context}\n\n質問: {question}"
    )
    initial_chain = initial_prompt | llm | StrOutputParser()

    # 追加文書に基づき既存の回答を改善するプロンプト
    refine_prompt = PromptTemplate.from_template(
        "これまでの回答:\n{existing_answer}\n\n"
        "追加の文書:\n{context}\n\n"
        "この情報を参考に、元の回答を改善してください。\n\n質問: {question}"
    )
    refine_chain = refine_prompt | llm | StrOutputParser()

    # refine処理の実行
    answer = initial_chain.invoke({
        "context": docs[0].page_content,
        "question": query
    })

    for doc in docs[1:]:
        answer = refine_chain.invoke({
            "existing_answer": answer,
            "context": doc.page_content,
            "question": query,
            "return_intermediate_steps": True
        })

    return answer


# 実行

## レポート作成 stuff

In [ ]:
# レポート作成機関範囲
months = get_ym_list(start_ym, n_term)

# 回答本文を格納するリストを初期化
reports = []

# 対象国のマップを取得
name = area_dict.get(area)

# 期間分繰り返す
for month in months:

    print(f"***** {month} 処理開始 *****")

    # プロンプト作成
    query = make_prompt(month, area)

    # 検索用クエリ作成
    search_query_jp = make_search_query_jp(month, area)

    # 検索用クエリ作成
    search_query_en = make_search_query_en(month, area)

    # chromaDBディレクトリの設定
    persist_dir = f"chroma_{name[2]}_db_{month}"

    #
    vectordb, urls = build_vectorstore_from_query(search_query_jp, search_query_en, persist_dir)


    result = answer_from_vectordb(query, vectordb)


    reports.append((month, result, sorted(set(urls))))


save_report_docx(reports, area, chain="stuff")


***** 2025年04月 処理開始 *****
日本語 取得URL件数 : 10
英語 取得URL件数 : 10
読み込み失敗: https://www.reuters.com/business/boj-keep-rates-steady-cut-growth-forecasts-2025-04-30/ -> 401 Client Error: HTTP Forbidden for url: https://www.reuters.com/business/boj-keep-rates-steady-cut-growth-forecasts-2025-04-30/
読み込み失敗: https://www.imf.org/en/News/Articles/2025/04/01/pr25084-japan-imf-executive-board-concludes-2025-article-iv-consultation-with-japan -> HTTPSConnectionPool(host='www.imf.org', port=443): Read timed out. (read timeout=10)
読み込み失敗: https://www.reuters.com/world/japan/japans-core-inflation-accelerates-complicates-bojs-rate-path-2025-04-17/ -> 401 Client Error: HTTP Forbidden for url: https://www.reuters.com/world/japan/japans-core-inflation-accelerates-complicates-bojs-rate-path-2025-04-17/
***** テキスト分割　開始 *****
トークン合計 118646 / 最大 128000
トークン合計 33397 / 最大 128000
***** ベクトル化　開始 *****
***** 2025年05月 処理開始 *****
日本語 取得URL件数 : 10
英語 取得URL件数 : 10
読み込み失敗: https://www.reuters.com/markets/asia/japans-core-inf

## レポート作成 map_reduce

In [ ]:
# レポート作成機関範囲
months = get_ym_list(start_ym, n_term)

# 回答本文を格納するリストを初期化
reports_map = []

# 対象国のマップを取得
name = area_dict.get(area)

# 期間分繰り返す
for month in months:

    print(f"***** {month} 処理開始 *****")

    # プロンプト作成
    query = make_prompt(month, area)

    # 検索用クエリ作成
    search_query_jp = make_search_query_jp(month, area)

    # 検索用クエリ作成
    search_query_en = make_search_query_en(month, area)

    # chromaDBディレクトリの設定
    persist_dir = f"chroma_{name[2]}_db_{month}"

    #
    vectordb, urls = build_vectorstore_from_query(search_query_jp, search_query_en, persist_dir)


    result_map = answer_from_vectordb_map(query, vectordb)

    reports_map.append((month, result_map, sorted(set(urls))))


save_report_docx(reports_map, area, chain="map_reduce")


***** 2025年04月 処理開始 *****
日本語 取得URL件数 : 10
英語 取得URL件数 : 10
読み込み失敗: https://jp.reuters.com/economy/bank-of-japan/HIK7JCJK3ZIRVC2F32E5HXGZAI-2025-05-27/ -> HTTPSConnectionPool(host='jp.reuters.com', port=443): Read timed out. (read timeout=10)
読み込み失敗: https://www.reuters.com/business/boj-keep-rates-steady-cut-growth-forecasts-2025-04-30/ -> 401 Client Error: HTTP Forbidden for url: https://www.reuters.com/business/boj-keep-rates-steady-cut-growth-forecasts-2025-04-30/
読み込み失敗: https://tradingeconomics.com/japan/interest-rate -> HTTPSConnectionPool(host='tradingeconomics.com', port=443): Read timed out. (read timeout=10)
読み込み失敗: https://www.imf.org/en/News/Articles/2025/04/01/pr25084-japan-imf-executive-board-concludes-2025-article-iv-consultation-with-japan -> HTTPSConnectionPool(host='www.imf.org', port=443): Read timed out. (read timeout=10)
読み込み失敗: https://www.reuters.com/world/japan/japans-core-inflation-accelerates-complicates-bojs-rate-path-2025-04-17/ -> 401 Client Error: HTTP Forb

## レポート作成 refine

In [ ]:
# レポート作成機関範囲
months = get_ym_list(start_ym, n_term)

# 回答本文を格納するリストを初期化
reports_ref = []

# 対象国のマップを取得
name = area_dict.get(area)

# 期間分繰り返す
for month in months:

    print(f"***** {month} 処理開始 *****")

    # プロンプト作成
    query = make_prompt(month, area)

    # 検索用クエリ作成
    search_query_jp = make_search_query_jp(month, area)

    # 検索用クエリ作成
    search_query_en = make_search_query_en(month, area)

    # chromaDBディレクトリの設定
    persist_dir = f"chroma_{name[2]}_db_{month}"

    #
    vectordb, urls = build_vectorstore_from_query(search_query_jp, search_query_en, persist_dir)


    result_ref = answer_from_vectordb_ref(query, vectordb)

    reports_ref.append((month, result_ref, sorted(set(urls))))


save_report_docx(reports_ref, area, chain="refine")


***** 2025年04月 処理開始 *****
日本語 取得URL件数 : 10
英語 取得URL件数 : 10
読み込み失敗: https://www.reuters.com/business/boj-keep-rates-steady-cut-growth-forecasts-2025-04-30/ -> 401 Client Error: HTTP Forbidden for url: https://www.reuters.com/business/boj-keep-rates-steady-cut-growth-forecasts-2025-04-30/
読み込み失敗: https://www.imf.org/en/News/Articles/2025/04/01/pr25084-japan-imf-executive-board-concludes-2025-article-iv-consultation-with-japan -> HTTPSConnectionPool(host='www.imf.org', port=443): Read timed out. (read timeout=10)
読み込み失敗: https://www.reuters.com/world/japan/japans-core-inflation-accelerates-complicates-bojs-rate-path-2025-04-17/ -> 401 Client Error: HTTP Forbidden for url: https://www.reuters.com/world/japan/japans-core-inflation-accelerates-complicates-bojs-rate-path-2025-04-17/
***** テキスト分割　開始 *****
トークン合計 118662 / 最大 128000
トークン合計 33419 / 最大 128000
***** ベクトル化　開始 *****
***** 2025年05月 処理開始 *****
日本語 取得URL件数 : 10
英語 取得URL件数 : 10
読み込み失敗: https://www.reuters.com/markets/asia/japans-core-inf